In [3]:
#Takes a list L of lists (linear extensions)
#Returns a list of elements and a list of relations that defines the
#poset obtained by intersecting the linear extensions of L
#To obtain such order, one has to compute Poset(poset_from_linear_extensions(L))
import networkx as nx
from itertools import combinations

def poset_from_linear_extensions(L):
    # Build the based set
    elements = set()
    for ext in L:
        elements.update(ext)
    
    elements = sorted(elements)
    
    # Build the set of relations from the linear extensions in L
    order_relation = set()
    for ext in L:
        for i in range(len(ext)):
            for j in range(i + 1, len(ext)):
                order_relation.add((ext[i], ext[j]))
    
    # Verify that the relations are satisfied by all linear extensions in L
    common_relations = set(order_relation)
    for (a, b) in order_relation:
        for ext in L:
            if not (ext.index(a) < ext.index(b)):
                common_relations.discard((a, b))
                break
    
    return elements, common_relations

In [4]:
#Takes a tree order P that has a maximum or a minimum
#Returns a pair of linear extension that forms a realizer of P
#See Lemma 5.1
def extLinRootTree(P):
    X=P.dimension(certificate=True)
    if X[0]==1:
        Y=X[1]+X[1]
    if X[0]==2:
        Y=[X[1][0]]+[X[1][1]]
    return Y

In [5]:
#Takes a rooted tree order P with root a and an element b that covers a. (b can be the empty list [] if necessary)
#Returns a triplet of linear extensions that forms a realizer of P restricted to the elements c such that the path
#from a to c in the Hasse diagram of P starts with an up-step that is not on b (take b=[] if one does not want to
#omit such element)
#See Proposition 5.2.
def fp(P,a,b):
    if b==[]:
        B=[]
    else:
        B=P.principal_order_ideal(b)
    C=P.upper_covers(a)
    D=[]
    if len(C)!=0:
        for i in C:
            if i in B:
                D=P.principal_order_filter(i)
    E=P.list()
    for i in D:
        E.remove(i)
    Q=P.subposet(E)
    F=Q.principal_order_filter(a)
    if F==[a]:
        return [[a],[],[],[],[]]
    R=Q.subposet(F)
    L=extLinRootTree(R)
    T=[]
    for i in range(1,len(F)):
        T=T+[fm(P,L[0][i],a)]
    W=[]
    X=[]
    A=[]
    Y=[]
    Z=[]
    for i in range(len(T)):
        W=W+T[i][1]+T[i][0]
        X=X+T[L[0].index(L[1][i+1])-1][3]+T[L[0].index(L[1][i+1])-1][0]
        A=T[i][4]+A
        Y=Y+T[i][0]
        Z=T[i][2]+Z
    X=X+A
    return [[a],W,X,Y,Z] #These are [a, U_{a,1}, U_{a,2}, U⁺_a, U⁻_a] according to the notation of Proposition 5.2

In [6]:
#Takes a rooted tree order P with root a and an element b covered by a. (b can be the empty list [] if necessary)
#Returns a triplet of linear extensions that forms a realizer of P restricted to the elements c such that the path
#from a to c in the Hasse diagram of P starts with a down-step that is not on b (take b=[] if one does not want to
#omit such element)
#See Proposition 5.2.
def fm(P,a,b):
    if b==[]:
        B=[]
    else:
        B=P.principal_order_filter(b)
    C=P.lower_covers(a)
    D=[]
    if len(C)!=0:
        for i in C:
            if i in B:
                D=P.principal_order_ideal(i)
    E=P.list()
    for i in D:
        E.remove(i)
    Q=P.subposet(E)
    F=Q.principal_order_ideal(a)
    if F==[a]:
        return [[a],[],[],[],[]]
    R=Q.subposet(F)
    L=extLinRootTree(R)
    T=[]
    for i in range(len(F)-1):
        T=T+[fp(P,L[0][i],a)]
    W=[]
    X=[]
    A=[]
    Y=[]
    Z=[]
    for i in range(len(T)):
        W=W+T[i][0]+T[i][1]
        X=X+T[L[0].index(L[1][i])][0]+T[L[0].index(L[1][i])][3]
        A=T[i][4]+A
        Y=Y+T[i][0]
        Z=T[i][2]+Z
    X=A+X
    return [[a],W,X,Y,Z] #These are [a, D_{a,1}, D_{a,2}, D⁻_a, D⁺_a] according to the notation of Proposition 5.2

In [7]:
#Takes a rooted tree order P with root a
#Returns the list [[a], U⁻_a + D_{a,1} = I⁻_a, U⁺_a, D⁻_a, U_{a,1} + D⁺_a = I⁺_a, D_{a,2}, U_{a,2}].
#See Corollary 5.3
def f(P,a):
    A=fp(P,a,[])
    B=fm(P,a,[])
    U=A[4]+B[1]    #U⁻_a + D_{a,1} = I⁻_a
    V=A[3]         #U⁺_a
    W=B[3]         #D⁻_a
    X=A[1]+B[4]    #U_{a,1} + D⁺_a = I⁺_a
    Y=B[2]         #D_{a,2}
    Z=A[2]         #U_{a,2}
    return [[a],U,V,W,X,Y,Z]

In [8]:
#Takes a tree poset P and a saturated chain L of P.
#Returns the list [A⁻, A⁺, B⁻, B⁺, C] where A(X)=A⁻ X A⁺ and B(Z)=B⁻ Z B⁺.
#See Corollary 5.4
def h(P,L):
    if len(L)==0:
        return [[],[],[],[],[]]
    elif len(L)==1:
        A=[f(P,L[0])]
    else:
        A=[]
        for i in range(len(L)):
            if i==0:
                T=P.principal_order_filter(L[i+1])
            elif i==len(L)-1:
                T=P.principal_order_ideal(L[i-1])
            else:
                T=P.principal_order_filter(L[i+1])+P.principal_order_ideal(L[i-1])
            S=P.list()
            for j in T:
                S.remove(j)
            Q=P.subposet(S)
            A=A+[f(Q,L[i])]
    V=[]    #A⁻
    W=[]    #A⁺
    X=[]    #B⁻
    Y=[]    #B⁺
    Z=[]    #C
    for i in range(len(A)):
        V=A[i][1]+V
        W=W+A[i][0]+A[i][2]
        X=X+A[i][3]+A[i][0]
        Y=A[i][4]+Y
        Z=Z+A[i][5]+A[i][0]+A[i][6]
    return [V,W,X,Y,Z]

In [9]:
#Takes a list L of 4n pairs.
#The first n pairs are rooted tree poset [T,x], where the x's will be the minimal elements of C_n.
#The following n pairs are rooted tree poset [T,z], where the z's will be the maximal elements of C_n.
#The 2n last pairs are pairs [T,C] of a tree poset T and a saturated chain C of T; these will be the tree grafted to the covers.
#Returns a triplet of linear extension of a unicycle poset that is a realizer of this poset.
#See Propositions 8.1 to 8.4 depending on n.
def ExtLinTotal(L):
    if len(L)%4!=0:
        return "Watch out, not the rigth number of trees!"
    n=len(L)/4
    M=[]
    for i in range(2*n):
        M=M+[f(L[i][0],L[i][1])]
    for i in range(2*n,4*n):
        M=M+[h(L[i][0],L[i][1])]
    X=[]
    Y=[]
    Z=[]
    if n==1:
        X=M[2][0]+M[0][3]+M[0][0]+M[2][1]+M[3][2]+M[1][5]+M[1][0]+M[1][6]+M[3][3]+M[0][4]
        Y=M[1][1]+M[3][0]+M[0][5]+M[0][0]+M[0][6]+M[3][1]+M[2][2]+M[1][0]+M[1][2]+M[2][3]
        Z=M[0][1]+M[0][0]+M[0][2]+M[2][4]+M[3][4]+M[1][3]+M[1][0]+M[1][4]
    elif n==2:
        X=M[4][0]+M[0][3]+M[0][0]+M[4][1]+M[1][3]+M[1][0]+M[5][2]+M[2][5]+M[2][0]+M[2][6]+M[5][3]+M[6][2]+M[7][2]+M[3][5]+M[3][0]+M[3][6]+M[7][3]+M[6][3]+M[0][4]+M[1][4]
        Y=M[3][1]+M[2][1]+M[5][0]+M[6][0]+M[1][5]+M[1][0]+M[1][6]+M[6][1]+M[5][1]+M[7][0]+M[0][5]+M[0][0]+M[0][6]+M[7][1]+M[3][0]+M[3][2]+M[4][2]+M[2][0]+M[2][2]+M[4][3]
        Z=M[0][1]+M[0][0]+M[0][2]+M[7][4]+M[1][1]+M[1][0]+M[1][2]+M[4][4]+M[6][4]+M[3][3]+M[3][0]+M[3][4]+M[5][4]+M[2][3]+M[2][0]+M[2][4]
    elif n==3:
        X=M[3][1]+M[1][1]+M[6][0]+M[0][3]+M[0][0]+M[6][1]+M[1][0]+M[1][2]+M[7][2]+M[3][0]+M[3][2]+M[7][3]+M[11][4]+M[8][4]+M[9][0]+M[2][3]+M[2][0]+M[9][1]+M[4][3]+M[4][0]+M[4][4]+M[10][2]+M[5][5]+M[5][0]+M[5][6]+M[10][3]+M[2][4]+M[0][4]
        Y=M[4][1]+M[8][0]+M[1][5]+M[1][0]+M[1][6]+M[8][1]+M[2][1]+M[2][0]+M[2][2]+M[9][2]+M[4][0]+M[4][2]+M[9][3]+M[7][4]+M[10][4]+M[11][0]+M[0][5]+M[0][0]+M[0][6]+M[11][1]+M[5][3]+M[5][0]+M[6][2]+M[3][5]+M[3][0]+M[3][6]+M[6][3]+M[5][4]
        Z=M[5][1]+M[0][1]+M[0][0]+M[0][2]+M[10][0]+M[2][5]+M[2][0]+M[2][6]+M[10][1]+M[11][2]+M[5][0]+M[5][2]+M[11][3]+M[6][4]+M[9][4]+M[7][0]+M[1][3]+M[1][0]+M[7][1]+M[8][2]+M[4][5]+M[4][0]+M[4][6]+M[8][3]+M[3][3]+M[3][0]+M[3][4]+M[1][4]
    else:
        X=M[n][1]+M[1][1]+M[2*n][0]+M[0][3]+M[0][0]+M[2*n][1]+M[1][0]+M[1][2]+M[2*n+1][2]+M[n][0]+M[n][2]+M[2*n+1][3]+M[4*n-1][4]+M[2*n+2][4]+M[2*n+3][0]+M[2][3]+M[2][0]+M[2*n+3][1]+M[n+1][3]+M[n+1][0]+M[n+1][4]
        Y=M[n+1][1]+M[2*n+2][0]+M[1][5]+M[1][0]+M[1][6]+M[2*n+2][1]+M[n+2][1]+M[2][1]+M[2][0]+M[2][2]+M[2*n+3][2]+M[n+1][0]+M[n+1][2]+M[2*n+3][3]+M[2*n+1][4]
        Z=M[2*n-1][1]+M[0][1]+M[0][0]+M[0][2]+M[4*n-2][0]+M[n-1][5]+M[n-1][0]+M[n-1][6]+M[4*n-2][1]+M[4*n-1][2]+M[2*n-1][0]+M[2*n-1][2]+M[4*n-1][3]+M[2*n][4]
        A=[]
        B=[]
        C=[]
        for i in range(3,n):
            A=A+M[2*n+2*i-2][4]+M[2*n+2*i-1][0]+M[i][3]+M[i][0]+M[2*n+2*i-1][1]+M[n+i-1][3]+M[n+i-1][0]+M[n+i-1][4]+M[i-1][4]
        for i in range(3,n-1):
            B=B+M[n+i][1]+M[i][1]+M[i][0]+M[i][2]+M[2*n+2*i-2][2]+M[2*n+2*i-1][2]+M[n+i-1][0]+M[n+i-1][2]+M[2*n+2*i-1][3]+M[2*n+2*i-2][3]
        for i in range(n-1,2,-1):
            C=C+M[2*n+2*i-1][4]+M[2*n+2*i-2][0]+M[i-1][5]+M[i-1][0]+M[i-1][6]+M[2*n+2*i-2][1]+M[n+i-1][5]+M[n+i-1][0]+M[n+i-1][6]
        X=X+A+M[4*n-2][2]+M[2*n-1][5]+M[2*n-1][0]+M[2*n-1][6]+M[4*n-2][3]+M[n-1][4]+M[0][4]
        Y=Y+B+M[n-1][1]+M[n-1][0]+M[n-1][2]+M[4*n-4][2]+M[4*n-3][2]+M[2*n-2][0]+M[2*n-2][2]+M[4*n-3][3]+M[4*n-4][3]+M[4*n-2][4]+M[4*n-1][0]+M[0][5]+M[0][0]+M[0][6]+M[4*n-1][1]+M[2*n-1][3]+M[2*n-1][0]+M[2*n][2]+M[n][5]+M[n][0]+M[n][6]+M[2*n][3]+M[2*n-1][4]
        Z=Z+C+M[2*n+3][4]+M[2*n+1][0]+M[1][3]+M[1][0]+M[2*n+1][1]+M[2*n+2][2]+M[n+1][5]+M[n+1][0]+M[n+1][6]+M[2*n+2][3]+M[n][3]+M[n][0]+M[n][4]+M[1][4]
    return [X,Y,Z]

In [12]:
#Takes two integers n and r.
#Returns a list of 4n random trees with at most r vertices.
#The 2n first trees have at least one vertex, the 2n last can be empty.
#Each vertex in this family of trees is labelled differently.
def TreesBuilder(n,r):
    if n==1:
        S=[randint(1,r) for i in range(2*n)]+[randint(1,r) for i in range(2*n)]
    else:
        S=[randint(1,r) for i in range(2*n)]+[randint(0,r) for i in range(2*n)]
    T=[]
    k=0
    for i in range(4*n):
        T=T+[graphs.RandomTree(S[i])]
        T[i].relabel(lambda x: x+k)
        k=k+S[i]
    return T

#Takes a list of 4n labelled trees.
#Returns a list of 4n tree posets according to the labelling of each trees.
def PosetsBuilder(T):
    L=[]
    for i in range(len(T)):
        t=T[i].edges(labels=False)
        rels=[]
        for j in range(len(t)):
            rels=rels+[[t[j][0],t[j][1]]]
        elem=T[i].vertices()
        L=L+[[Poset([elem,rels], cover_relations = True)]]
    return L

import random

#Takes a list of 4n tree posets.
#Returns a list of 4n entries.
#The first 2n entries are rooted tree posets with a randomly chosen root.
#The last 2n entries are pairs [T,C] for T a tree poset and C a randomly chosen saturated chain of T.
def VertexChoice(L):
    n=len(L)/4
    for i in range(2*n):
        L[i]=L[i]+[random.choice(L[i][0].list())]
    for i in range(2*n,4*n):
        if L[i][0].cardinality()==0:
            L[i]=L[i]+[[]]
        else:
            H=L[i][0].hasse_diagram()
            L[i]=L[i]+[random.choice(sum([H.all_paths(x,y) for x in H for y in L[i][0].principal_order_filter(x)],[]))]
    return L

#Takes two integers n and r.
#Returns a list of 4n pairs.
#The first 2n pairs are of the shape [T,r] for T a random rooted tree poset with at least one element and most r element, and r a randomly chosen root of T.
#The last 2n pairs are of the shape [T,C] for T a tree poset and C a randomly chosen saturated chain of T.
def UnicycleBuilder(n,r):
    P=VertexChoice(PosetsBuilder(TreesBuilder(n,r)))
    return P

In [13]:
#Takes a list as of what UnicyclBuider returns (with length 4n).
#Returns a unicycle poset made of the vertex-grafting of the first n rooted tree posets on the minimal elements of C_n,
#the vertex-grafting of the following n rooted tree posets on the maximal elements of C_n and,
#the cover-grafting of the last 2n tree posets on the covers of C_n.
def PosetVerif(L):
    if len(L)%4!=0:
        return "Watch out, not the right number of trees!"
    n=len(L)/4
    rels=[]
    elem=[]
    for i in range(4*n):
        elem=elem+L[i][0].list()
        rels=rels+L[i][0].cover_relations()
    if n==1:
        rels=rels+[[L[0][1],L[2][1][0]],[L[2][1][-1],L[1][1]],[L[0][1],L[3][1][0]],[L[3][1][-1],L[1][1]]]
    else:
        for i in range(n):
            if len(L[2*n+2*i][1])==0:
                rels=rels+[[L[i][1],L[i+n][1]]]
            else:
                rels=rels+[[L[i][1],L[2*n+2*i][1][0]],[L[2*n+2*i][1][-1],L[i+n][1]]]
        if len(L[4*n-1][1])==0:
            rels=rels+[[L[0][1],L[2*n-1][1]]]
        else:
            rels=rels+[[L[0][1],L[4*n-1][1][0]],[L[4*n-1][1][-1],L[2*n-1][1]]]
        for i in range(n-1):
            if len(L[2*n+2*i+1][1])==0:
                rels=rels+[[L[i+1][1],L[n+i][1]]]
            else:
                rels=rels+[[L[i+1][1],L[2*n+2*i+1][1][0]],[L[2*n+2*i+1][1][-1],L[n+i][1]]]
    return Poset((elem, rels), cover_relations = True)

In [14]:
#Takes two integers n and r.
#Build a family F of 4n trees with UnicycleBuilder.
#Returns a triplet (R,P,F) where F is the family of trees T_x, T_z and T_p,
#R is build using ExtLinTotal on the family F and P is build using PosetVerif also on F.
#The two poest P and R should be isomorphic.
def Builders(n,r):
    F=UnicycleBuilder(n,r)
    R=Poset(poset_from_linear_extensions(ExtLinTotal(F)))
    P=PosetVerif(F)
    return [F,R,P]

In [15]:
A=Builders(5,10)

#Verify if the two ways of building unicycle posets gives the same poset
A[1].is_isomorphic(A[1])

#Show in a javascript the two orders (they should be the same)
A[1].hasse_diagram().show('js', link_distance=50)
A[2].hasse_diagram().show('js', link_distance=50)

AttributeError: 'list' object has no attribute 'is_isomorphic'

In [17]:
#En fixant les entiers r, n et p, on teste des graphes aléatoires.
#Pour chaque 0<k<n+1, on teste p ordres obtenus en greffant des arbres aléatoires de taille au plus r sur la couronne C_k.
#Takes three integers r, n and p.
#We build p random unicycle posets using both the grafting method and the three linear extensions of Propositions
#and verify if in each case the two ways of building the posets gives the same poset.
#If the two ways gave the same poset for each 0<k<p, it returns "It worked!"
#Else it returns the list of posets that are not isomorphic with the family of trees used to build the posets
def Tester(r,n,p):
    X=[]
    for i in range(p):
        L=UnicycleBuilder(n,r)
        P=PosetVerif(L)
        R=Poset(poset_from_linear_extensions(ExtLinTotal(L)))
        if P.is_isomorphic(R)==False:
            X=X+[[L,R,P]]
    if len(X)==0:
        return "It worked for %p posets with underlying crown C_%n with grafted trees of size at most %r" % (
    if len(X)!=0:
        return X

In [18]:
Tester(20,10,50)

'It worked!'